In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)


def VGG19_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10,
                                                                    stratify=y_train)
    print(len(X_train))
    model = keras.applications.VGG19(include_top=True, weights=None, input_shape=(227,227,3), pooling=max, classes=4)
    model.compile(loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'],
                  optimizer=keras.optimizers.SGD(learning_rate=0.01))
    model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_validation, y_validation))
    evaluation = []
    evaluation.append(model.evaluate(X_train, y_train, batch_size=16))
    evaluation.append(model.evaluate(X_test, y_test, batch_size=16))
    return evaluation

In [2]:
image_size = 227

groups_folder_path = '../Data/resultVecsFigs/AAFT_0/'

X_s = []

y = []
# auto_corr       hadamard        midAng     striping_color      striping_midAng_color
for label in ['Bi', 'Tri', 'Hammer', 'Rvcurl']:

    for feature in ['hadamard']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            #print(f)
            for filename in f:
                #print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])
                
                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                elif label == 'Tri':
                    y.append(1)
                elif label == 'Hammer':
                    y.append(2)
                else:
                    y.append(3)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

vgg = []
for i in range(10):
    vgg.append(VGG19_model(X, y))

(227, 227, 3)
240
172
Epoch 1/10
11/11 [==============================] - 22s 1s/step - loss: 1.3864 - accuracy: 0.2168 - val_loss: 1.3855 - val_accuracy: 0.2500
Epoch 2/10
11/11 [==============================] - 2s 165ms/step - loss: 1.3858 - accuracy: 0.3016 - val_loss: 1.3853 - val_accuracy: 0.2500
Epoch 3/10
11/11 [==============================] - 2s 165ms/step - loss: 1.3855 - accuracy: 0.2685 - val_loss: 1.3846 - val_accuracy: 0.4000
Epoch 4/10
11/11 [==============================] - 2s 165ms/step - loss: 1.3851 - accuracy: 0.2687 - val_loss: 1.3842 - val_accuracy: 0.5000
Epoch 5/10
11/11 [==============================] - 2s 165ms/step - loss: 1.3844 - accuracy: 0.3743 - val_loss: 1.3842 - val_accuracy: 0.4000
Epoch 6/10
11/11 [==============================] - 2s 165ms/step - loss: 1.3840 - accuracy: 0.3637 - val_loss: 1.3833 - val_accuracy: 0.4000
Epoch 7/10
11/11 [==============================] - 2s 167ms/step - loss: 1.3838 - accuracy: 0.3758 - val_loss: 1.3825 - val_acc

11/11 [==============================] - 2s 166ms/step - loss: 1.3863 - accuracy: 0.2977 - val_loss: 1.3858 - val_accuracy: 0.2500
Epoch 3/10
11/11 [==============================] - 2s 171ms/step - loss: 1.3862 - accuracy: 0.2057 - val_loss: 1.3858 - val_accuracy: 0.2500
Epoch 4/10
11/11 [==============================] - 2s 175ms/step - loss: 1.3857 - accuracy: 0.2851 - val_loss: 1.3856 - val_accuracy: 0.2500
Epoch 5/10
11/11 [==============================] - 2s 171ms/step - loss: 1.3859 - accuracy: 0.3128 - val_loss: 1.3854 - val_accuracy: 0.5000
Epoch 6/10
11/11 [==============================] - 2s 171ms/step - loss: 1.3857 - accuracy: 0.3021 - val_loss: 1.3852 - val_accuracy: 0.5000
Epoch 7/10
11/11 [==============================] - 2s 166ms/step - loss: 1.3863 - accuracy: 0.2479 - val_loss: 1.3851 - val_accuracy: 0.5000
Epoch 8/10
11/11 [==============================] - 2s 166ms/step - loss: 1.3851 - accuracy: 0.4151 - val_loss: 1.3851 - val_accuracy: 0.4000
Epoch 9/10
11/11 

In [3]:
for data in vgg:
    print(data[0][0])

1.379067301750183
1.3818076848983765
1.3814177513122559
1.38160240650177
1.378251314163208
1.3842887878417969
1.3815743923187256
1.3806850910186768
1.3833906650543213
1.380959153175354


In [4]:
for data in vgg:
    print(data[1][1])

0.5416666865348816
0.5208333134651184
0.5
0.7083333134651184
0.5
0.5
0.6041666865348816
0.8333333134651184
0.25
0.6041666865348816


In [5]:
'''
image_size = 227

groups_folder_path = '../Data/resultVecsFigs/AAFT_5/'

X_s = []

y = []
# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw
for label in ['Bi', 'Tri']:

    for feature in ['hadamard']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            # print(f)
            for filename in f:
                # print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                else:
                    y.append(1)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

vgg = []
for i in range(10):
    vgg.append(VGG19_model(X, y))
    for data in vgg:
for d in data:
    for j in d:
        print(j)
'''

'\nimage_size = 227\n\ngroups_folder_path = \'../Data/resultVecsFigs/AAFT_5/\'\n\nX_s = []\n\ny = []\n# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw\nfor label in [\'Bi\', \'Tri\']:\n\n    for feature in [\'hadamard\']:\n\n        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):\n            # print(f)\n            for filename in f:\n                # print(groups_folder_path + label + "/" + feature + "/" + filename)\n                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)\n                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])\n\n                X_s.append(img / 256)\n\n                if label == \'Bi\':\n                    y.append(0)\n                else:\n                    y.append(1)\n\nprint(X_s[0].shape)\n# print(X_s)\nprint(len(y))\nX = np.array(X_s)\ny = np.array(y)\n\nvgg = []\nfor i in ra

In [6]:
'''
image_size = 227

groups_folder_path = '../Data/resultVecsFigs/AAFT_10/'

X_s = []

y = []
# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw
for label in ['Bi', 'Tri']:

    for feature in ['hadamard']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            # print(f)
            for filename in f:
                # print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                else:
                    y.append(1)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

vgg = []
for i in range(10):
    vgg.append(VGG19_model(X, y))
for data in vgg:
    for d in data:
        for j in d:
            print(j)
'''

'\nimage_size = 227\n\ngroups_folder_path = \'../Data/resultVecsFigs/AAFT_10/\'\n\nX_s = []\n\ny = []\n# auto_corr   hadamard     midAng    striping_color   striping_bw     striping_midAng_color     striping_midAng_bw\nfor label in [\'Bi\', \'Tri\']:\n\n    for feature in [\'hadamard\']:\n\n        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):\n            # print(f)\n            for filename in f:\n                # print(groups_folder_path + label + "/" + feature + "/" + filename)\n                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)\n                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])\n\n                X_s.append(img / 256)\n\n                if label == \'Bi\':\n                    y.append(0)\n                else:\n                    y.append(1)\n\nprint(X_s[0].shape)\n# print(X_s)\nprint(len(y))\nX = np.array(X_s)\ny = np.array(y)\n\nvgg = []\nfor i in r